In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD
from keras.utils import plot_model
import multiprocessing as mp
import os


#Obtener los datos


#Preprocesar los datos


#Crear el modelo


#Entrenar el modelo


#Evaluar el modelo


In [5]:
def load_and_resize_image(img_path, size):
    img = imread(img_path)
    img_resized = resize(img, size, anti_aliasing=True)
    return img_resized

df = pd.read_csv('D:/Sebastian/Proyecto/Machine_Learning/Dataset/train.csv')

# Verificar si los datos preprocesados ya existen
preprocessed_data_path = 'preprocessed_data.npz'
if os.path.exists(preprocessed_data_path):
    data = np.load(preprocessed_data_path)
    X_raw = data['X_raw']
    y = data['y']
else:
    # Cargar y preprocesar las imágenes
    X_raw = []
    size = (28, 28)
    img_paths = [f'D:/Sebastian/Proyecto/Machine_Learning/Dataset/train/train/{row["ID"]}_{row["location"]}.jpg' for _, row in df.iterrows()]

    # Usar multiprocessing para cargar y redimensionar las imágenes en paralelo
    with mp.Pool(mp.cpu_count()) as pool:
        X_raw = pool.starmap(load_and_resize_image, [(img_path, size) for img_path in img_paths])

    X_raw = np.array(X_raw) / 255.0
    y = df['level'].values

    np.savez(preprocessed_data_path, X_raw=X_raw, y=y)

X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.10, random_state=42, stratify=y)
if X_train.ndim == 3:
    X_train = X_train[..., None]
    X_test = X_test[..., None]
    print(X_train.shape, X_test.shape)

def show_results(model, log):
    loss, acc = model.evaluate(X_test, y_test, batch_size=512, verbose=False)
    print(f'Loss     = {loss:.4f}')
    print(f'Accuracy = {acc:.4f}')
    val_loss = log.history['val_loss']
    val_acc = log.history['val_accuracy']
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(val_loss, label='test')
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('loss')
    ax2.plot(log.history['accuracy'], label='train')
    ax2.plot(val_acc, label='test')
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('accuracy')
    for ax in axes:
        ax.legend()

In [ ]:
#@title Visualizando el `dataset`
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    idx = np.random.choice(X.shape[0], 1)[0]
    plt.title('id:{} val:{}'.format(idx, y[idx] ))
    plt.imshow(-X[idx], cmap='gray')
    plt.axis('off')
plt.show()